In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import time



In [2]:
data_dir = '../Data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [3]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = models.resnet50(pretrained=True)
#model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
#model.load_state_dict(torch.load('resnetCoco.h5'))
#model = torch.load(os.path.dirname(os.path.abspath('D:\\8th Semester\\resnetCoco')))
model = models.resnet50(num_classes=2)
model.load_state_dict(torch.load("../resnetCocoPytorch.pt"), strict=False)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
#print(model) 
model.fc = nn.Sequential(nn.Linear(2048, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
#Adam => same as stochastic gradient descent + Momentum
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to(device);
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
epochs = 1
steps = 0
runningLoss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Utilizing my GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logModelOutput = model.forward(inputs)
        loss = criterion(logModelOutput, labels)
        #back prop + optimizing the parameters
        loss.backward()
        optimizer.step()

        runningLoss += loss.item()
        
        if steps % print_every == 0:
            testLoss = 0
            accuracy = 0
            #turn off dropout and other stuff helpful for training
            evaluationStart = time.time()
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logModelOutput = model.forward(inputs)
                    batchLoss = criterion(logModelOutput, labels)
                    
                    testLoss += batchLoss.item()
                    
                    # Calculate accuracy
                    probabilityOutput = torch.exp(logModelOutput)
                    topProbability, predictedClass = probabilityOutput.topk(1, dim=1)
                    predictedClass = torch.squeeze(predictedClass) 
                    equals = (predictedClass == labels)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                          
            evaluationEnd = time.time()      
            print(f"Epoch {steps}.. "
                  f"Train loss: {runningLoss/print_every:.3f}.. "
                  f"Test loss: {testLoss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}"
                  f" Eval Time Per Image: {(evaluationEnd-evaluationStart)/len(testloader):.3f}"
                 )
            running_loss = 0
            model.train()

Epoch 5.. Train loss: 0.763.. Test loss: 0.784.. Test accuracy: 0.512 Eval Time Per Image: 0.795
Epoch 10.. Train loss: 1.514.. Test loss: 0.507.. Test accuracy: 0.690 Eval Time Per Image: 0.708
Epoch 15.. Train loss: 2.091.. Test loss: 0.431.. Test accuracy: 0.816 Eval Time Per Image: 0.741
Epoch 20.. Train loss: 2.624.. Test loss: 0.332.. Test accuracy: 0.934 Eval Time Per Image: 0.714
Epoch 25.. Train loss: 3.088.. Test loss: 0.272.. Test accuracy: 0.934 Eval Time Per Image: 0.716
Epoch 30.. Train loss: 3.540.. Test loss: 0.235.. Test accuracy: 0.926 Eval Time Per Image: 0.748


KeyboardInterrupt: 

In [7]:
#torch.save(model.state_dict(), 'poop.h5')